Найдите список всех самолетов, находящихся сейчас в воздухе. (Имеется в виду на момент выполнения домашки)

Возможно вам поможет это: https://aviationstack.com/documentation

0. Импорт необходимых программных библиотек и определение переменных

In [1]:
import requests
import json


#AvStack_access_key = 'c4a3d5cf155522d2c5bf376683118f0c' // Blocked
AvStack_access_key = 'f442611685a69e93498e353f024bf5ed'
AvStack_API_requests_url = 'http://api.aviationstack.com/v1/'
params = {'access_key': AvStack_access_key}

allAirports = []
allFlightsForMyAC = []

allFlightsCount = 0

allAirlinesCount = 0
allAirportsCount = 0

dataCountPerRequest = 100

# Code Excample
# https://aviationstack.com/documentation#:~:text=Code%20Example%20%2D%20Python

1. Получим число аэропортов в базе данных сайта aviationstack.com

In [2]:
params.update({'limit': 1})
api_result = requests.get(AvStack_API_requests_url + 'airports', params)

# api_result.status_code // HTTP код состояния
# api_result.content // Текст полученного ответа
# api_result.headers // Словарь полученных HTTP заголовков
if api_result.status_code == 200:
    api_response = api_result.json()
    api_response = api_response['pagination']['total']
    if type(api_response) == int:
        allAirportsCount = api_response
        print ('Из API ответа получили актуальное количество аэропортов allAirportsCount = ', allAirportsCount)
else:
    print ('GET Обращение к ' + AvStack_API_requests_url + 'airports было неуспешным. HTTP код ответа: ' + str(api_result.status_code))
    if api_result.status_code == 429:
        print ('Ошибка вызвана превышением лимита бесплатной учетной записи')
    

Из API ответа получили актуальное количество аэропортов allAirportsCount =  6705


2. Получим информацию об актуальном числе рейсов

In [3]:
api_result = requests.get(AvStack_API_requests_url + 'flights', params)
if api_result.status_code == 200:
    api_response = api_result.json()
    api_response = api_response['pagination']['total']
    if type(api_response) == int:
        allFlightsCount = api_response
        print ('Из API ответа получили актуальное количество рейсов allFlightsCount = ', allFlightsCount)
else:
    print ('GET Обращение к ' + AvStack_API_requests_url + 'flights было неуспешным. HTTP код ответа: ' + str(api_result.status_code))
    if api_result.status_code == 429:
        print ('Ошибка вызвана превышением лимита бесплатной учетной записи')

Из API ответа получили актуальное количество рейсов allFlightsCount =  339065


3. Получим информацию по аэропортам - ограниченная часть данных (300) с сайта {AvStack_API_requests_url + 'airports'}

In [4]:

if allAirportsCount > 0:
    params.update({'limit': dataCountPerRequest})
    #cycleNeeded = 1 + allAirportsCount // dataCountPerRequest
    cycleNeeded = 3 # !
    
    for skip in range (0, dataCountPerRequest * cycleNeeded, dataCountPerRequest):
        #currParams['offset'] = skip
        params.update({'offset': skip})
        api_result = requests.get(AvStack_API_requests_url + 'airports', params).json()
        # print(api_result)
        # просмотр JSON структуры - https://jsonformatter.curiousconcept.com/#
        allAirports += api_result['data']
    
    # Получили структуру данных с информацией об аэропортах allAirports. Данные взяты не в полном объеме
    # в силу ограничений бесплатной учетной записи

4. Получим информацию по авиакомпаниям - ограниченная часть данных (300) с сайта {AvStack_API_requests_url + 'airlines'}

In [5]:
params = {'access_key': AvStack_access_key, 'limit': 1}
api_result = requests.get(AvStack_API_requests_url + 'airlines', params)
if api_result.status_code == 200:
    api_response = api_result.json()
    api_response = api_response['pagination']['total']
    if type(api_response) == int:
        allAirlinesCount = api_response
        print ('Из API ответа получили актуальное количество аэропортов allAirlinesCount = ', allAirlinesCount)
    

Из API ответа получили актуальное количество аэропортов allAirlinesCount =  13131


5. Число рейсов в воздухе.
Для запроса данных будет взята авиокомпания "United Airlines" ("airline_name")

In [27]:
params = {'access_key': AvStack_access_key, 'limit': 1}
params.update({'flight_status': 'active', 'airline_name': 'United Airlines'})

api_result = requests.get(AvStack_API_requests_url + 'flights', params)
if api_result.status_code == 200:
    api_response = api_result.json()
    api_response = api_response['pagination']['total']
    if type(api_response) == int:
        CurrAirlineFlightsCount = api_response
print (CurrAirlineFlightsCount)

476


На момент времени 22.09 Мск в дату 06.12.2021 у авиакомпании "United Airlines" 521 рейс имеет статус "active" (воздушное судно в воздухе)

Список рейсов в воздухе

In [28]:
allFlightsForMyAC = [] #Обнуляш


#params.update({'limit': 1})
params.update({'limit': dataCountPerRequest})

#cycleNeeded = 1
cycleNeeded = 1 + CurrAirlineFlightsCount // dataCountPerRequest

for skip in range (0, dataCountPerRequest * cycleNeeded, dataCountPerRequest):
    cycleNeeded -= 1
    if cycleNeeded == 0:
        params.update({'limit': CurrAirlineFlightsCount % dataCountPerRequest})
    params['offset'] = skip
    #params.update({'offset': skip})
    api_result = requests.get(AvStack_API_requests_url + 'airports', params).json()
    allFlightsForMyAC += api_result['data']

In [30]:
allFlightsForMyAC

[{'id': '1',
  'gmt': '-10',
  'airport_id': '1',
  'iata_code': 'AAA',
  'city_iata_code': 'AAA',
  'icao_code': 'NTGA',
  'country_iso2': 'PF',
  'geoname_id': '6947726',
  'latitude': '-17.05',
  'longitude': '-145.41667',
  'airport_name': 'Anaa',
  'country_name': 'French Polynesia',
  'phone_number': None,
  'timezone': 'Pacific/Tahiti'},
 {'id': '2',
  'gmt': '10',
  'airport_id': '2',
  'iata_code': 'AAB',
  'city_iata_code': 'AAB',
  'icao_code': 'YARY',
  'country_iso2': 'AU',
  'geoname_id': '7730796',
  'latitude': '-26.7',
  'longitude': '141.04167',
  'airport_name': 'Arrabury',
  'country_name': 'Australia',
  'phone_number': None,
  'timezone': 'Australia/Brisbane'},
 {'id': '3',
  'gmt': '2',
  'airport_id': '3',
  'iata_code': 'AAC',
  'city_iata_code': 'AAC',
  'icao_code': 'HEAR',
  'country_iso2': 'EG',
  'geoname_id': '6297289',
  'latitude': '31.133333',
  'longitude': '33.75',
  'airport_name': 'El Arish International Airport',
  'country_name': 'Egypt',
  'phon

In [31]:
len(allFlightsForMyAC)

476

In [33]:
import pandas as pd
pd.json_normalize(allFlightsForMyAC)

,id,gmt,airport_id,iata_code,city_iata_code,icao_code,country_iso2,geoname_id,latitude,longitude,airport_name,country_name,phone_number,timezone
0,1,-10,1,AAA,AAA,NTGA,PF,6947726,-17.05,-145.41667,Anaa,French Polynesia,None,Pacific/Tahiti
1,2,10,2,AAB,AAB,YARY,AU,7730796,-26.7,141.04167,Arrabury,Australia,None,Australia/Brisbane
2,3,2,3,AAC,AAC,HEAR,EG,6297289,31.133333,33.75,El Arish International Airport,Egypt,None,Africa/Cairo
3,4,1,4,AAE,AAE,DABB,DZ,2570559,36.821392,7.811857,Les Salines,None,None,Africa/Algiers
4,5,-5,5,AAF,AAF,KAAF,US,4146153,29.733334,-84.98333,Apalachicola Regional,United States,None,America/New_York
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,626,3,626,BEI,BEI,HABE,ET,7730583,9.391667,34.533333,Beica,Ethiopia,None,Africa/Addis_Ababa
472,627,8,627,BEJ,BEJ,WALK,ID,7668667,2.166667,117.7,Kalimaru,Indonesia,None,Asia/Makassar
473,629,-3,629,BEL,BEL,SBBE,BR,6300600,-1.389865,-48.480003,Val De Cans,Brazil,None,America/Belem
474,630,2,630,BEN,BEN,HLLB,LY,6297330,32.085423,20.26489,Benina International,Libya,None,Africa/Tripoli
